In [1]:
!pip install sqlalchemy==1.3.9


  Using cached SQLAlchemy-1.3.9-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.35
    Uninstalling SQLAlchemy-2.0.35:
      Successfully uninstalled SQLAlchemy-2.0.35
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
jupyterhub 4.1.5 requires SQLAlchemy>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.


In [2]:
!pip install ipython-sql

  Using cached SQLAlchemy-2.0.35-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached SQLAlchemy-2.0.35-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9


In [3]:
%load_ext sql

In [4]:
import csv, sqlite3
con = sqlite3.connect('my_data1.db')
cur = con.cursor


In [5]:
!pip install -q pandas

In [6]:
%sql sqlite:///my_data1.db

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

In [8]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [9]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

In [10]:
-- TASK 1
%sql SELECT DISTINCT LAUNCH_SITE FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [11]:
-- TASK 2
%sql SELECT * FROM SPACEXTABLE WHERE LAUNCH_SITE LIKE 'CCA%' LIMIT 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [44]:
%%sql SELECT SUM(PAYLOAD_MASS__KG_) AS TOTAL_PAYLOAD_MASS 
FROM SPACEXTABLE 
WHERE CUSTOMER = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


TOTAL_PAYLOAD_MASS
45596


In [43]:
%%sql SELECT AVG(PAYLOAD_MASS__KG_) as AVG_PAYLOAD_MASS
from SPACEXTABLE
where BOOSTER_VERSION = 'F9 v1.1';

 * sqlite:///my_data1.db
Done.


AVG_PAYLOAD_MASS
2928.4


In [37]:
%%sql SELECT MIN(Date) AS FIRST_SUCCESSFUL_GROUP_LANDING
from SPACEXTABLE
where "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


FIRST_SUCCESSFUL_GROUP_LANDING
2015-12-22


In [38]:
%%sql SELECT DISTINCT BOOSTER_VERSION
FROM SPACEXTABLE
WHERE "Landing_Outcome" = 'Success (drone ship)'
    AND PAYLOAD_MASS__KG_ > 4000
    AND PAYLOAD_MASS__KG_ < 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [39]:
%%sql SELECT SUM(CASE WHEN MISSION_OUTCOME like "%Success%" THEN 1 ELSE 0 END) AS SUCCESSFUL_MISSIONS,
    SUM(CASE WHEN MISSION_OUTCOME LIKE '%Failure%' then 1 else 0 end) as FAILED_MISSIONS
FROM SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


SUCCESSFUL_MISSIONS,FAILED_MISSIONS
100,1


In [35]:
%%sql SELECT DISTINCT BOOSTER_VERSION 
FROM SPACEXTABLE 
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [46]:
%%sql SELECT 
  substr(Date, 6, 2) AS MONTH,
  "Landing_Outcome",
  BOOSTER_VERSION,
  LAUNCH_SITE
FROM SPACEXTABLE
WHERE substr(Date, 1, 4) = '2015'
  AND "Landing_Outcome" = 'Failure (drone ship)';

 * sqlite:///my_data1.db
Done.


MONTH,Landing_Outcome,Booster_Version,Launch_Site
01,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


In [47]:
%%sql SELECT 
  "Landing _Outcome",
  COUNT(*) AS OUTCOME_COUNT
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing _Outcome"
ORDER BY OUTCOME_COUNT DESC;

 * sqlite:///my_data1.db
Done.


"""Landing _Outcome""",OUTCOME_COUNT
Landing _Outcome,31
